In [1]:
import numpy as np

In [2]:
"""
Input format: [[a,b]...] where a = remainder and b = modulus

"""
inp = [[1,2],[2,3],[4,5]]

In [3]:
# Calculates gcd of a pair of numbers
def gcd_euclidean_number_pair(a,b):
    if a == 0:
        return b;
    elif b == 0:
        return a;
    if a < 0:
        a = -a
    if b < 0:
        b = -b
    
    if a == b:
        return a;
    elif a > b:
        return gcd_euclidean_number_pair(a%b,b)
    elif a < b:
        return gcd_euclidean_number_pair (a,b%a)
    
# Calculates gcd of an array of numbers
def gcd_euclidean(inp):
    # take gcd of a pair and repeat the process
    gcd_num_pair = inp[0]
    for i in range(0,len(inp)-1):
        a = gcd_num_pair
        b = inp[i + 1]
        gcd_num_pair = gcd_euclidean_number_pair(a,b)
    gcd = gcd_num_pair
    return gcd

In [4]:
"""Compute the coefficients of Bezout's identity. a and b are 2 co-prime numbers. """
def extended_euclidean(a,b):
    
    if (gcd_euclidean([a,b]) != 1):
        raise ValueError("GCD of numbers not equal to 1. No integer solutions are possible")
    a_org = a
    b_org = b
    a_prev = a
    b_prev = b
    coeffs = [(a,b)]
    while (b!=0):
        a = b_prev
        b = a_prev%b_prev
        coeffs.append((a,b))
        a_prev = a
        b_prev = b  
    x = 1
    y = 0
    for i in range(0,len(coeffs)):
        a_prev = coeffs[len(coeffs) - i - 1][0]
        b_prev = coeffs[len(coeffs) - i - 1][1]
        if b_prev == 0:
            continue
        x_prev = y
        y_prev = x - (a_prev//b_prev)*y
        x = x_prev
        y = y_prev 
    # mod operation in python gives result with matching sign of denominator.
    if (a_org*x + b_org*y == -1):
        x = -x
        y = -y
    return x,y

"""gcd of a and b must be 1 for modular inverse to exist. b is the modulus"""
def calculate_modular_inverse(a,b):
    if(b<=0):
        raise ValueError("Modulus cannot be o or negative")
    inv = extended_euclidean(a,b)[0]
    if(b>0 and inv<0):
        inv = inv + b
        
    return inv

In [6]:
"""
x = a1(mod m1)
x = a2(mod m2)
x = a3(mod m3)

where m1,m2 and m3 are co-prime

then x = a1*alpha + a2*beta + a3*gamma 

such that

alpha = 1(mod m1) = 0(mod m2) = 0(mod m3)
beta =  0(mod m1) = 1(mod m2) = 0(mod m3)
gamma = 0(mod m1) = 0(mod m2) = 1(mod m3)

So alpha = Qa * m2*m3, beta = Qb * m1*m3 and gamma = Qg * m1*m2

Now since alpha = 1(mod m1) => Qa*m2*m3 = 1(mod m1) => Qa = (m2*m3)^-1 (mod m1). (m2*m3)^-1 exists because gcd(m2*m3,m1) = 1

Similarly Qb and Qg can be determined and then alpha, beta, gamma and  finally x.

Note that Qa = k*m1 + (m2*m3)^-1. So this function returns 2 arguments i.e. lcm of moduli and the remainder.

"""

"""
Simpler Explanation:

Suppose 
x = 1(mod 2) - (1)
x = 2(mod 3) - (2)
x = 4(mod 5) - (3)

from eqn (1) 
x = 2t+1 - (4)

Substituting eqn (4) in eqn (2)
2t+1 = 2(mod 3) => 2t = 1(mod 3) => t = 2(mod 3) => t = 3s + 2 - (5)

Substituting eqn (5) in eqn(4) 
x = 6s + 5 - (6)

Substituting eqn(6) in eqn (3)
6s + 5 = 4(mod 5) => 6s = -1(mod 5) = 4(mod 5) => s = 4(mod 5) => s = 5k + 4 - (7)

Substituting eqn (7) in eqn (6)
x = 30k + 29 - (8)

Now 30 = 2*3*5 = product of moduli. Eqn (8) satisfies eqn (1),(2) and (3) for different values of k.

"""
def chinese_remainder_implementation(inp):
    inp = np.array(inp)
    inp_shape = np.shape(inp)
    if(len(inp_shape)!=2):
        raise ValueError("Input shape must be 2 dimensionsal.")
    if(inp_shape[1]!=2):
        raise ValueError("Axis 1 of input must have length of 2")
    # Check if moduli are coprime
    moduli = inp[:,1]
    remainders = inp[:,0]
    if(gcd_euclidean(moduli)!=1):
        raise ValueError("Moduli are not co-prime")
    qs = []
    moduli_prods = []
    for i in range(0,len(moduli)):
        moduli_prod=1;
        for j in range(0,len(moduli)):
            if(i==j):
                continue
            else:
                moduli_prod = moduli_prod*moduli[j]
        moduli_prods.append(moduli_prod)
        qs.append(calculate_modular_inverse(moduli_prod,moduli[i]))
    alphas = np.multiply(moduli_prods,qs)
    x = np.sum(np.multiply(alphas,remainders))
    return (np.prod(moduli),x%np.prod(moduli))
        
chinese_remainder_implementation(inp)

(30, 29)